<a href="https://colab.research.google.com/github/jinijinhee/delta/blob/main/pyspark_%EB%9E%9C%EB%8D%A4%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8%2C_%EA%B2%B0%EC%A0%95%ED%8A%B8%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [15]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pyspark as spark
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
import glob
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import SQLContext
sc = SparkContext()
sqlContext = SQLContext(sc)


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [16]:
df = pd.read_csv('./drive/MyDrive/df.csv')
df.head()

spark_df = sqlContext.createDataFrame(df)

In [25]:
from pyspark.ml.feature import VectorAssembler

# 1. Train/Test 데이터로 분할
train_df, test_df = spark_df.randomSplit(weights=[0.8, 0.2], seed=42)

# 2. Feature Vector로 만들기
ftr_columns = ['max_rpm','s20', 's30', 's40', 's50', 's60', 's70', 's80','s90','s100','s110', 's120','s130']
vec_assembler = VectorAssembler(inputCols=ftr_columns, outputCol='features') #features에는 하나의 칼럼.

train_ftr_vec = vec_assembler.transform(train_df)
test_ftr_vec = vec_assembler.transform(test_df)  # 테스트 데이터에 대해도 동일한 객체 사용

print('# 학습 원본 데이터:')
train_df.limit(5).show()

print('# Feature Vectorization 후 학습 데이터:')
train_ftr_vec.limit(5).show()

# 학습 원본 데이터:
+-----+----------+-----------+----------+-----------+-----------+---+---+---+---+---+---+---+---+----+----+----+----+----+----+------------------------------+----+-------------+------------------+------------------+------------------+-------------------+-------------------+--------+------------------+-----------------+------+--------------------+--------------------+
|index|     s_lat|      s_lng|     e_lat|      e_lng|    max_rpm|s20|s30|s40|s50|s60|s70|s80|s90|s100|s110|s120|s130|s140|s150|                          path| fee|duration(min)|     root_dist(km)|      med_est_dist|              diff|                pc1|                pc2|clusters|          time_sum|        avg_speed|target|                   s|                   e|
+-----+----------+-----------+----------+-----------+-----------+---+---+---+---+---+---+---+---+----+----+----+----+----+----+------------------------------+----+-------------+------------------+------------------+------------------+-------------

#랜덤포레스트

In [26]:
from pyspark.ml.classification import RandomForestClassifier

# 모델 정의
dt_clf = RandomForestClassifier(featuresCol='features', labelCol='target', numTrees=10)

# 학습
dt_model = dt_clf.fit(train_ftr_vec)

# 학습, 테스트 데이터에 대해 예측
train_pred = dt_model.transform(train_ftr_vec)
test_pred = dt_model.transform(test_ftr_vec)

In [27]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='target', metricName='accuracy')

train_acc = evaluator.evaluate(train_pred)
test_acc = evaluator.evaluate(test_pred)

print('Train Accuracy:', train_acc)
print('Test Accuracy:', test_acc)

Train Accuracy: 0.868270332187858
Test Accuracy: 0.8298969072164949


#결정트리

In [30]:
from pyspark.ml.classification import DecisionTreeClassifier

# 모델 정의
dt_clf = DecisionTreeClassifier(featuresCol='features', labelCol='target', maxDepth=5)

# 학습
dt_model = dt_clf.fit(train_ftr_vec)

# 학습, 테스트 데이터에 대해 예측
train_pred = dt_model.transform(train_ftr_vec)
test_pred = dt_model.transform(test_ftr_vec)

In [31]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='target', metricName='accuracy')

train_acc = evaluator.evaluate(train_pred)
test_acc = evaluator.evaluate(test_pred)

print('Train Accuracy:', train_acc)
print('Test Accuracy:', test_acc)

Train Accuracy: 0.8972890416189385
Test Accuracy: 0.8316151202749141
